In [ ]:
import os, sys

# 1. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 2. Change directory to project root
project_path = "/content/drive/MyDrive/medical-image-cnn"
%cd $project_path

# 3. Add project root to Python path
if project_path not in sys.path:
    sys.path.append(project_path)

# 4. Ensure __init__.py files exist in src/ packages
folders = [
    "src",
    "src/classification",
    "src/segmentation",
    "src/detection"
]
for folder in folders:
    init_file = os.path.join(folder, "__init__.py")
    if not os.path.exists(init_file):
        with open(init_file, "w") as f:
            f.write("# init\n")
        print(f"Created: {init_file}")

# 5. Check structure
print("SRC contents:", os.listdir(os.path.join(project_path, "src")))


Mounted at /content/drive
/content/drive/MyDrive/medical-image-cnn
SRC contents: ['classification', 'segmentation', 'detection', 'utils.py', '__init__.py', '__pycache__']


In [ ]:
!pip install tensorflow keras matplotlib seaborn scikit-learn opencv-python


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

from src.classification.data_loader import get_classification_generators
from src.classification.model_builder import build_mobilenet_classifier
from src.utils import plot_training
import numpy as np # Added numpy import

# Load Data
# Point data_dir to the root of the dataset where train, val, and test folders are located
data_dir = "/content/drive/MyDrive/medical-image-cnn/datasets/brain_mri"
img_size = (128,128)   # smaller images = faster
batch_size = 16

train_gen, val_gen, test_gen = get_classification_generators(
    data_dir=data_dir,
    img_size=img_size,
    batch_size=batch_size
)

# Get the classes from the training generator
train_classes = list(train_gen.class_indices.keys())
num_classes = len(train_classes)
print("Classes:", train_gen.class_indices)

# Filter test_gen to only include classes present in train_gen
# This assumes get_classification_generators returns a DirectoryIterator
if hasattr(test_gen, 'classes'):
    # Recreate class_indices for the test set based on available classes
    test_class_indices = {cls: i for i, cls in enumerate(sorted(list(set([test_gen.filenames[i].split('/')[0] for i in range(len(test_gen.filenames))]))))}

    test_indices = [i for i in range(len(test_gen.filenames)) if test_gen.filenames[i].split('/')[0] in train_classes]

    # Create a new generator with filtered indices
    from tensorflow.keras.utils import Sequence
    class FilteredTestDataGenerator(Sequence):
        def __init__(self, original_generator, indices, class_indices):
            self.generator = original_generator
            self.indices = indices
            self.batch_size = original_generator.batch_size
            self.class_indices = class_indices # Use the filtered class indices
            self.classes = np.array([self.class_indices[original_generator.filenames[i].split('/')[0]] for i in indices]) # Update classes attribute

        def __len__(self):
            return int(np.ceil(len(self.indices) / self.batch_size))

        def __getitem__(self, idx):
            batch_indices_in_original = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
            # Get the batch using the original generator's method
            # This might require accessing protected members, adjust if necessary based on the actual generator implementation
            batch_x, batch_y_original = self.generator._get_batches_of_transformed_samples(batch_indices_in_original)

            # If original_generator yields one-hot encoded labels, convert back to integer
            if batch_y_original.ndim > 1:
                batch_y_original = np.argmax(batch_y_original, axis=1)

            # Map original class indices to filtered class indices
            batch_y_filtered = np.array([self.class_indices[list(self.generator.class_indices.keys())[original_label]] for original_label in batch_y_original])

            # One-hot encode filtered labels if needed by the model
            # Assuming the model expects one-hot encoding
            num_filtered_classes = len(self.class_indices)
            batch_y_one_hot = np.zeros((len(batch_y_filtered), num_filtered_classes))
            batch_y_one_hot[np.arange(len(batch_y_filtered)), batch_y_filtered] = 1

            return batch_x, batch_y_one_hot


    # Create class_indices for the filtered test set
    filtered_test_class_labels = sorted(list(set([test_gen.filenames[i].split('/')[0] for i in test_indices])))
    filtered_test_class_indices = {cls: i for i, cls in enumerate(filtered_test_class_labels)}


    test_gen = FilteredTestDataGenerator(test_gen, test_indices, filtered_test_class_indices)
    print(f"Filtered test set to {len(test_indices)} images with classes from training set.")
    # Update num_classes to reflect the number of classes in the training set
    num_classes = len(train_classes)
    print(f"Number of classes for model: {num_classes}")

Found 13888 images belonging to 4 classes.
Found 3470 images belonging to 4 classes.
Found 4333 images belonging to 4 classes.
Classes: {'glioma_tumor': 0, 'meningioma_tumor': 1, 'normal': 2, 'pituitary_tumor': 3}
Filtered test set to 4333 images with classes from training set.
Number of classes for model: 4
